In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

In [9]:
df = pd.read_csv("Portland_Maps_Assessor_Data.csv")
clean_df = df.dropna()
clean_df.to_csv("cleaned_portland_data.csv", index=False)



C:\Users\josep\AppData\Local\Temp\ipykernel_12036\2132570383.py:1: DtypeWarning: Columns (2,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Portland_Maps_Assessor_Data.csv")


In [20]:
df = pd.read_csv("cleaned_portland_data.csv")

# Filter useful columns
features = ['square_feet', 'year_built', 'latitude', 'longitude', 'zip_code_string', 'neighborhood']
target = 'market_value'

# Drop rows with missing target or features
df = df.dropna(subset=features + [target])

# One-hot encode categorical features
df_encoded = pd.get_dummies(df[features])

# Set up input (X) and output (y)
X = df_encoded
y = df[target]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Print predicted vs actual values
for real, pred in zip(y_test[:10], y_pred[:10]):  # adjust [:10] to show more or fewer rows
    print(f"Actual: ${real:,.2f} \t Predicted: ${pred:,.2f}")
    

Actual: $593,630.00 	 Predicted: $798,783.24
Actual: $553,120.00 	 Predicted: $595,737.39
Actual: $615,530.00 	 Predicted: $754,966.03
Actual: $680,220.00 	 Predicted: $927,402.51
Actual: $375,320.00 	 Predicted: $407,405.06
Actual: $276,120.00 	 Predicted: $442,251.17
Actual: $478,480.00 	 Predicted: $697,285.50
Actual: $2,835,610.00 	 Predicted: $625,006.31
Actual: $348,560.00 	 Predicted: $278,643.02
Actual: $530,430.00 	 Predicted: $488,869.08


In [21]:
train_data = X_train.copy()
train_data[target] = y_train

# Save the training data to a new CSV file
train_data.to_csv("training_data.csv", index=False)

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Load the cleaned data
df = pd.read_csv("cleaned_portland_data.csv")

# Filter to only include rows where city is Portland and address does not contain 'TRUNCATED'
df = df[(df['city'].str.upper() == 'PORTLAND') & (~df['address'].str.upper().str.contains('TRUNCATED'))]

# Define features and target
features = ['square_feet', 'year_built', 'latitude', 'longitude', 'zip_code_string', 'neighborhood']
target = 'market_value'

# Drop rows with missing values in features or target
df = df.dropna(subset=features + [target])

# Remove rows where market value is 0
df = df[df['market_value'] != 0]

# One-hot encode categorical features
X_encoded = pd.get_dummies(df[features])
y = df[target]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=1)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Create error dataframe
error = y_test - y_pred
error_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred,
    'Error': error,
    'Abs_Error': error.abs()
})

# Filter out predictions where actual value is 0
error_df = error_df[error_df['Actual'] != 0]

# Sample 10 random rows
sampled_errors = error_df.sample(10, random_state=1)

# Get original test data (non-encoded)
original_test_data = df.loc[X_test.index]
sampled_rows = original_test_data.loc[sampled_errors.index]

# Add Actual and Predicted values
sampled_rows = sampled_rows.copy()
sampled_rows['Actual'] = sampled_errors['Actual'].round(2)
sampled_rows['Predicted'] = sampled_errors['Predicted'].round(2)

# Display relevant information
pd.set_option('display.float_format', '{:,.2f}'.format)
print("\n10 Random Predictions (Portland only, no TRUNCATED addresses):")
print(sampled_rows[['address', 'neighborhood', 'city', 'zip_code', 'latitude', 'longitude', 'Actual', 'Predicted']])



10 Random Predictions (Portland only, no TRUNCATED addresses):
                           address        neighborhood      city  zip_code  \
45839           1623 SE MALDEN ST    SELLWOOD-MORELAND  PORTLAND 97,202.00   
204609   6616 NE KILLINGSWORTH ST                CULLY  PORTLAND 97,218.00   
151568           4323 SE 12TH AVE             BROOKLYN  PORTLAND 97,202.00   
166700       4915 WI/ SE 17TH AVE             BROOKLYN  PORTLAND 97,202.00   
866      0837 SW PALATINE HILL RD         COLLINS VIEW  PORTLAND 97,219.00   
142490           4054 SE BOISE ST   CRESTON-KENILWORTH  PORTLAND 97,202.00   
60578            1914 NE 78TH AVE        MADISON SOUTH  PORTLAND 97,213.00   
160005        4627 N MARYLAND AVE             OVERLOOK  PORTLAND 97,217.00   
159721  4621-4623 N MINNESOTA AVE             OVERLOOK  PORTLAND 97,217.00   
218686          7325 N FOWLER AVE          ARBOR LODGE  PORTLAND 97,217.00   

        latitude  longitude        Actual     Predicted  
45839      45.47   

In [27]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Assuming you have already trained the model and have y_test (actual values) and y_pred (predicted values)
# If you haven't defined them, you can follow the previous code to do that

# Calculate the absolute error for each prediction
error = y_test - y_pred

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics with formatted output (no scientific notation)
print(f"Mean Absolute Error (MAE): {mae:,.2f}")
print(f"R² Score: {r2:.4f}")

# Create a DataFrame to examine both actual and predicted values with error
comparison_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred,
    'Error': error,
    'Abs_Error': error.abs()
})

# Format the numbers to avoid scientific notation
comparison_df['Actual'] = comparison_df['Actual'].apply(lambda x: f"{x:,.2f}")
comparison_df['Predicted'] = comparison_df['Predicted'].apply(lambda x: f"{x:,.2f}")
comparison_df['Error'] = comparison_df['Error'].apply(lambda x: f"{x:,.2f}")
comparison_df['Abs_Error'] = comparison_df['Abs_Error'].apply(lambda x: f"{x:,.2f}")

# Sort by the absolute error to see where the model made the largest mistakes
comparison_df_sorted = comparison_df.sort_values(by='Abs_Error', ascending=False)

# Display the top 5 largest errors
print("\nTop 5 samples with the largest errors:")
print(comparison_df_sorted.head(5))


Mean Absolute Error (MAE): 468,373.95
R² Score: 0.5263

Top 5 samples with the largest errors:
             Actual     Predicted        Error   Abs_Error
344      811,370.00  1,810,768.86  -999,398.86  999,398.86
24525          0.00    999,363.33  -999,363.33  999,363.33
80071  1,732,330.00  2,731,444.74  -999,114.74  999,114.74
65797    841,810.00  1,840,823.15  -999,013.15  999,013.15
72036    476,360.00    475,361.12       998.88      998.88
